In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras.datasets import mnist
import keras

from sklearn.metrics import confusion_matrix
import seaborn as sns #; sns.set_theme()

KeyboardInterrupt: 

## Load dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
fig, axs = plt.subplots(3, 3, figsize = (10, 10))
plt.gray()
for i, ax in enumerate(axs.flat):
    ax.imshow(x_train[i])
    ax.axis('off')
    ax.set_title('Number {}'.format(y_train[i]))
plt.show()

### Define model

In [ ]:
batch_size = 64
# Each MNIST image batch is a tensor of shape (batch_size, 28, 28).
# Each input sequence will be of size (28, 28) (height is treated like time).
input_dim = 28

units = 64
output_size = 10  # labels are from 0 to 9

# Build the RNN model
def build_model(allow_cudnn_kernel=True):
    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel,
    # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
    if allow_cudnn_kernel:
        # The LSTM layer with default options uses CuDNN.
        lstm_layer = keras.layers.LSTM(units, input_shape=(None, input_dim))
    else:
        # Wrapping a LSTMCell in a RNN layer will not use CuDNN.
        lstm_layer = keras.layers.RNN(
            keras.layers.LSTMCell(units), input_shape=(None, input_dim)
        )
    model = keras.models.Sequential(
        [
            lstm_layer,
            keras.layers.BatchNormalization(),
            keras.layers.Dense(output_size),
        ]
    )
    return model

In [ ]:
model = build_model(allow_cudnn_kernel=True)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)

model.fit(
    x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=20
)

### Model Behaviors 

In [ ]:
history = model.history

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.title('model train vs validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc='best')
plt.show()

### Predictions

In [ ]:
labels_prediction = model.predict(x_test)

In [ ]:
labels_pred = []
for x in labels_prediction:
    labels_pred.append(np.argmax(x))

In [ ]:
fig, axs = plt.subplots(3, 3, figsize = (10, 10))
plt.gray()
for i, ax in enumerate(axs.flat):
    ax.imshow(x_test[i])
    ax.axis('off')
    ax.set_title('Number predicted: {}'.format(labels_pred[i]))
plt.show()

In [ ]:
def copy(array):
    new_array = []
    for i in range(len(array)):
        new_line = [0]*len(array[i])
        for j in range(len(array[i])):
            new_line[j] = array[i][j]
        new_array.append(new_line)
    return new_array

def copyLine(arr):
    newLine = []
    for i in range(len(arr)):
        newLine.append(arr[i])
    return newLine

def pourcentage_ligne(matrice):
    new_matrice = copy(matrice)
    for i in range(len(matrice)):
        sum_ = np.sum(matrice[i])
        for j in range(len(matrice[i])):
            new_matrice[i][j] = 100* matrice[i][j]/sum_
    return new_matrice

In [ ]:
cf_matrix = confusion_matrix(y_test, labels_pred)
cf_matrix_pct_line = pourcentage_ligne(cf_matrix)

In [ ]:
plt.figure(figsize=(15,12))
ax = sns.heatmap(cf_matrix, annot=True, fmt="d", linewidth=1, cmap='Blues')
ax.invert_yaxis()
plt.title('Confusion Matrix\n', fontsize = 18) 
plt.xlabel('\nPredicted labels', fontsize = 13) 
plt.ylabel('True labels\n', fontsize = 13) 
plt.show()

plt.figure(figsize=(15,12))
ax = sns.heatmap(cf_matrix_pct_line, annot=True, fmt=".2f", linewidth=1, cmap='Blues')
ax.invert_yaxis()
for t in ax.texts: t.set_text(t.get_text() + " %")
plt.title('Confusion Matrix - percentage per line\n', fontsize = 18) 
plt.xlabel('\nPredicted labels', fontsize = 13) 
plt.ylabel('True labels\n', fontsize = 13) 
plt.show()

In [ ]:
nb_total_err = 0
for i in range(len(cf_matrix)):
    for j in range(len(cf_matrix[i])):
        if i != j:
            nb_total_err += cf_matrix[i][j]

In [ ]:
print(f"On the validation data, there are {nb_total_err} errors, over {len(x_test)} elements")
print(f"It represents a {round(nb_total_err/len(x_test), 2)} % error rate")

## Let's look at the mistakes

In [ ]:
indexes_of_errors = []
for i in range(len(y_test)):
    if y_test[i] != labels_pred[i]:
        indexes_of_errors.append(i)

In [ ]:
fig, axs = plt.subplots(5, 5, figsize = (15, 15))
plt.gray()
for j, ax in enumerate(axs.flat):
    i = indexes_of_errors[j]
    ax.imshow(x_test[i])
    ax.axis('off')
    ax.set_title(f'Predicted: {labels_pred[i]}, True: {y_test[i]}')
plt.show()

$ \text{
We're now going to take a closer look the the mistaken elements, } \newline
\text{check whether the true label was the second highest probability predicted by our model}
$

In [ ]:
def spot_of_true_label_in_prediction(true_label, prediction):
    value_pred_true_label = prediction[true_label]
    copyPrediction = copyLine(prediction)
    copyPrediction.sort(reverse=True)
    for i in range(len(copyPrediction)):
        if copyPrediction[i] == value_pred_true_label:
            return i
    return -1

In [ ]:
true_labels_prediction_spots = []
for index in indexes_of_errors:
    prediction_spot = spot_of_true_label_in_prediction(y_test[index], labels_prediction[index])
    true_labels_prediction_spots.append(prediction_spot)

In [ ]:
plt.figure(figsize=(10,7))
labels, counts = np.unique(true_labels_prediction_spots, return_counts=True)
#counts = counts / len(true_labels_prediction_spots)
plt.bar(labels, counts, align='center', color = 'black')
plt.gca().set_xticks(labels)
#plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.ylabel('Nummber of occurences\n')
plt.xlabel('Spot')
plt.title('Prediction spot of true label\n')
plt.show()

In [ ]:
indexes_of_major_errors = []
for i in range(len(true_labels_prediction_spots)):
    index = indexes_of_errors[i]
    if true_labels_prediction_spots[i] >= 3:
        indexes_of_major_errors.append(index)
        
print(f"There are {len(indexes_of_major_errors)} instances of major errors")

In [ ]:
def get_second_highest_pred(pred):
    max_, index_max = -10000, -1
    secondMax, index_sndMax = max_-1, -1
    for i in range(len(pred)):
        x = pred[i]
        if x > max_:
            secondMax, index_sndMax = max_, index_max
            max_, index_max = x, i
        elif x > secondMax:
            secondMax, index_sndMax = x, i
            
    return index_sndMax

In [ ]:
nb_per_line = 5

nb_shown = 0
while nb_shown < len(indexes_of_major_errors):
    fig, axs = plt.subplots(1, nb_per_line, figsize=(20, 3))
    for j in range(nb_per_line):
        if nb_shown < len(indexes_of_major_errors):
            i = indexes_of_errors[nb_shown]
            axs[j].imshow(x_test[i])
            axs[j].axis('off')
            snd_highest = get_second_highest_pred(labels_prediction[index])
            axs[j].set_title(f'Pred: {labels_pred[i]}, True: {y_test[i]}, 2nd Highest: {snd_highest}', fontsize=14)
            nb_shown += 1
        else:
            axs[j].axis('off')
    plt.show()